# **Lab 1: Matrix algorithms**
**Matthieu GANNE**

# **Abstract**


This lab is a summary of the basic function when someone wants to work on vector and matrix.This contains the implementation of the scalar product, the matrix-vector product, the matrix-matrix product and the computation of the euclidian norm and distance.

We implemeted also the Strassen algorithm for the matrix-matrix product.

#**About the code**

In [0]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Matthieu GANNE (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**


We will show you the implementation of classic mathematical functions that are used to manipulate scalars and matrix.


There is also the [Strassen algorithm](https://en.wikipedia.org/wiki/Strassen_algorithm ) for the Matrix-Matrix product.



# **Methods**

All the method start by checking the input then compute.We do not optimize the cache here. 

## ***Scalar Product***


$x * y = \sum_{i=1}^n x_i y_i, x,y \in \mathbb{R}^n$

### Function


In [0]:
def scalar_product(x,y):
    if(x.ndim != 1):
        return "x is not a vector with 1 dim"
    if(y.ndim != 1):
        return "y is not a vector with 1 dim"
    dim = x.shape[0]
    if dim != y.shape[0]:
        return "y and x do not have the same size"

    result = 0
    for i in range(dim):
        result += x[i]*y[i]

    return result


### Test


In [0]:
def test_scalar():
     for i in range(20):
        T1 = np.random.randint(10,size=250)
        T2 = np.random.randint(10,size=250)
        result = scalar_product(T1,T2)
        soluce = np.dot(T1,T2)
        assert np.array_equal(result,soluce) == True, "error Scalar"


## ***Matrix-Vector Product***

$Ax = b, A\in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n$

$b_{ij} =\sum_{j=1}^n a_{ij} x_j$

### Function


In [0]:
def matrix_vector_product(A,x):
    if(x.ndim != 1):
        return "x is not a vector with 1 dim"
    dimX = x.shape[0]
    if(A.shape[1] != dimX):
        return "size does not match for a product"

    dimA = A.shape[0]
    result = np.zeros(dimA)
    for i in range(dimA):
        for j in range(dimX):
            result[i] += A[i,j]*x[j]
    return result

### Test

In [0]:
def test_matrix_scalar():
    for i in range(20):
        x = np.random.randint(10,size=250)
        A = np.random.randint(10,size=(250,250))
        result = matrix_vector_product(A,x)
        soluce = np.dot(A,x)
        assert np.array_equal(result,soluce) == True, "error Scalar Matrix"


## **Matrix-Matrix Product**


$AB = C, A\in \mathbb{R}^{m\times l} B\in \mathbb{R}^{l\times n}$

$c_{ij} = \sum_{k=1}^l a_{ik}b_{kj}$

### Classic Function


In [0]:
def matrix_matrix_product(A,B):
    dimA0 = A.shape[0]
    dimA1 = A.shape[1]
    dimB1 = B.shape[1]
    
    if(dimA1 != B.shape[0]):
        return "size does not match for a product"
    result = np.zeros((dimA0,dimB1))
    for i in range(dimA0):
        for j in range(dimB1):
            for k in range(dimA1):
                result[i,j]+=A[i,k]*B[k,j]
    return result

### Test

In [0]:
def test_Matrix_Matrix():
  for i in range(20):
        T1 = np.random.randint(10,size=(7,125))
        T2 =  np.random.randint(10,size=(125,256))
        result = matrix_matrix_product(T1,T2)
        soluce = np.dot(T1,T2)
        assert np.array_equal(result,soluce) == True, "error Matrix_Matrix"

### Strassen


This is a bit complicated because we need to be careful about the padding since the size of the input matrix are not in the right format.

In [0]:
def strassen_product(A,B):
    dimA0 = A.shape[0]
    dimA1 = A.shape[1]
    dimB1 = B.shape[1]

   
    if(dimA1 != B.shape[0]):
        return "size does not match for a product"

    if(dimA0 == 1 or dimB1 == 1 or dimA1 == 1):
        return matrix_matrix_product(A,B)

    dimt0 =dimA0
    if dimA0%2 == 1:
        dimt0 = dimA0+1
    dimt1 =dimA1
    if dimA1%2 == 1:
        dimt1 = dimA1+1
    dimtB0 =dimA1
    if dimA1%2 == 1:
        dimtB0 = dimA1+1
    dimtB1 =dimB1
    if dimB1%2 == 1:
        dimtB1 = dimB1+1
        
    tempA = np.zeros((dimt0,dimt1))
    tempA[:dimA0,:dimA1] = A

    tempB = np.zeros((dimtB0,dimtB1))
    tempB[:dimA1,:dimB1] = B
    
    C = np.zeros((dimt0,dimtB1))
    k = int(dimt0/2)
    k1 = int(dimt1/2)
    k2 =  int(dimtB1/2)
    A11,A21,A12,A22 = tempA[:k,:k1], tempA[k:, :k1], tempA[:k, k1:], tempA[k:, k1:]
    B11,B21,B12,B22 = tempB[:k1,:k2], tempB[k1:, :k2], tempB[:k1, k2:], tempB[k1:, k2:]

    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10= B11 + B12

    P1 =  matrix_matrix_product(A11, S1)
    P2 =  matrix_matrix_product(S2, B22)
    P3 =  matrix_matrix_product(S3, B11)
    P4 =  matrix_matrix_product(A22, S4)
    P5 =  matrix_matrix_product(S5, S6)
    P6 =  matrix_matrix_product(S7, S8)
    P7 =  matrix_matrix_product(S9, S10)
    C[:k,:k2] = P5 + P4 - P2 + P6
    C[k:, :k2] = P3 + P4
    C[:k, k2:] = P1 + P2
    C[k:, k2:] = P5 + P1 - P3 - P7

    return C[:dimA0,:dimB1]

### Test

In [0]:
def test_Strassen():
    for i in range(20):
        T1 = np.random.randint(10,size=(7,125))
        T2 =  np.random.randint(10,size=(125,256))
        result = strassen_product(T1,T2)
        soluce = np.dot(T1,T2)
        assert np.array_equal(result,soluce) == True, "error Strassen"

### ***Euclidian norm***


$||x||_2 = ({x_1}^2 +...+{x_n}^2)^{1/2}$

## Function




In [0]:
def euclidian_norm(x):
    if(x.ndim != 1):
        return "x is not a vector with 1 dim"

    result = 0
    for v in x:
        result += v*v
    result = math.sqrt(result)
    return result

## Test

In [0]:
def test_Euclidian():
    for i in range(20):
        x = np.random.randint(10,size=250)
        result = euclidian_norm(x)
        soluce = np.linalg.norm(x)
        assert np.array_equal(result,soluce) == True, "error Euclidian Norm"

## Euclidian distance

$d(x,y) = ||x-y||_2$

### Function

We can see that we need to be careful here and do a copy of x. Because if we do not, we would modify x.

In [0]:
def euclidian_distance(x,y):
    if(x.ndim != 1):
        return "x is not a vector with 1 dim"
    if(y.ndim != 1):
        return "y is not a vector with 1 dim"
    dim = x.shape[0]
    if dim != y.shape[0]:
        return "y and x do not have the same size"

    result = np.copy(x)
    for i in range(dim):
        result[i] -= y[i];
    result = euclidian_norm(result)
    return result

### Test

In [0]:
def test_Euclidian_dist():
     for i in range(20):   
        x = np.random.randint(10,size=100)
        y = np.random.randint(10,size=100)
        result = euclidian_distance(x,y)
        soluce = np.linalg.norm(x-y)
        assert np.array_equal(result,soluce) == True, "error Euclidian dist"

# **Results**

We run the tests as we have shown before.


In [0]:
if __name__== "__main__":
    test_scalar()
    test_matrix_scalar()
    test_Matrix_Matrix()
    test_Strassen()
    test_Euclidian()
    test_Euclidian_dist()
    print "Test Succeed"

We could have added the tests that deal with borderline cases that are not in the right format.
We note in all cases that these implementations are not efficient.